<a href="https://colab.research.google.com/github/fatema373/NLP/blob/main/Text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install country_list

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 35.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import string
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer
from country_list import countries_for_language

In [ ]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Wikepidia API

In [ ]:
!pip install wikipedia-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import wikipediaapi
import random
def Loading_data(categories,n_docs):
    # print(categories)
    docs,labels=[],[]
    wikiapi = wikipediaapi.Wikipedia('en')
    for category in categories:
        #find all pages under the umbrella of this categry
        category_topics=wikiapi.page("Category:{}".format(category))#dict
        category_pages_names=category_topics.categorymembers.values()
        #extract page title from each page to get it from wikipediaapi later
        pages_title=[]
        for page_title in category_pages_names:
            pages_title.append(page_title.title)
        # randomly select n titles from pages_titles and save them
        count_docs=0
        while count_docs!=n_docs:
            page_title=random.sample(pages_title,1)
            pages_title.remove(page_title[0])
            page = wikiapi.page(page_title[0])
            if page.exists() and len(page.text.split())>100:
                docs.append(page.text.lower())
                labels.append(category)
                count_docs+=1
            else:
                print(page_title[0],' Not qualified')
    temp = list(zip(docs, labels))
    random.shuffle(temp)
    docs, labels = zip(*temp)
    return docs ,labels

In [ ]:
categories=['technology','Politics','History','Art','Geography','Economics','Education','Health','Science']
docs,labels= Loading_data([categories[1],categories[2]],n_docs=20)

In [ ]:
len(docs)

40

In [ ]:
dict_data=dict(zip(docs,labels))

In [ ]:
dict_data.values()

dict_values(['Politics', 'History', 'Politics', 'History', 'Politics', 'Politics', 'Politics', 'History', 'Politics', 'Politics', 'Politics', 'History', 'History', 'History', 'Politics', 'Politics', 'History', 'History', 'History', 'History', 'Politics', 'Politics', 'History', 'History', 'Politics', 'Politics', 'Politics', 'History', 'Politics', 'Politics', 'History', 'History', 'History', 'Politics', 'Politics', 'History', 'History', 'History', 'Politics', 'History'])

In [ ]:
df = pd.DataFrame(dict_data.items(), columns=['docs', 'label'])

In [ ]:
df

,docs,label
0,comparative politics is a field in political s...,Politics
1,this glossary of history is a list of definiti...,History
2,a substitute is a political candidate who is n...,Politics
3,the landscape crisis of the guia fortress refe...,History
4,a diplomatic mission or foreign mission is a g...,Politics
5,health security is a concept that encompasses ...,Politics
6,politics is the process by which groups of peo...,Politics
7,"a biography, or simply bio, is a detailed desc...",History
8,"a term of office, electoral term, or parliamen...",Politics
9,political modernization (also spelled as polit...,Politics


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   docs    40 non-null     object
 1   label   40 non-null     object
dtypes: object(2)
memory usage: 768.0+ bytes


# Text preprocessing

In [ ]:
'''
  The Steps we'll be following to Clean the given Docs
  1- Lower-casing
  2- Remove stopwords, special chars, numbers,months, countries
  3-take Tokens for each document given
  4- Limitization (get the origin for each word)
Text_Preprocessing class has one attribute: data (Dataframe)
'''
class Text_Preprocessing:
  def __init__(self,data):
    self.data= self.removing_trash(data)
    self.tokens=self.get_tokens(self.data.clean_text)
    self.limitizers=self.limitization(self.data.tokenization)

    '''
    Removing trash function is a functions takes 1 argument (docs) and
    return new col 'docs_without_stopwords'
    '''
  def remove_puctuations(self,text):
    punctuations = string.punctuation
    return text.translate(str.maketrans('','',punctuations))
  def remove_stopwords(self,text):
    STOPWORDS = set(stopwords.words('english'))
    return ' '.join(word for word in text.split() if word not in STOPWORDS)

  def remove_special_char(self,text):
    text = re.sub('[^a-zA-Z]',' ',text)
    text = re.sub('\s+',' ',text)
    return text

  def remove_countries(self,text):
    countries = dict(countries_for_language('en'))
    countries_Vals = [x.lower() for x in list(countries.values())]
    countries_Keys = [x.lower() for x in list(countries.keys())]
    countries_to_be_removed=countries_Vals+countries_Keys
    return ' '.join(word for word in text.split() if word not in (countries_to_be_removed))

  def remove_months(self,text):
    months=['january','february','march','april', 'may','june','july','august','september','october','november','december']
    return ' '.join(word for word in text.split() if word not in (months))

  def removing_trash(self,data):
    #remove punctiuation
    punctuations = string.punctuation
    data['clean_text']= data.docs.apply(lambda x: self.remove_puctuations(x))

    #remove stopswords
    data['clean_text'] = data['clean_text'].apply(lambda x: self.remove_stopwords(x))

    #remove Special-chars and numbers
    data['clean_text'] = data['clean_text'].apply(lambda x: self.remove_special_char(x))

    #remove countries and their abbriviations
    data['clean_text'] = data['clean_text'].apply(lambda x: self.remove_countries(x))

    #remove countries and their abbriviations
    data['clean_text'] = data['clean_text'].apply(lambda x: self.remove_countries(x))

    #remove months
    data['clean_text'] = data['clean_text'].apply(lambda x: self.remove_months(x))
    return data

  '''
  Second Step to take is get tokens for each document
  '''
  def tokenization(self,text):
    return nltk.word_tokenize(text) #return token for given document

  def get_tokens(self,docs):
    self.data['tokenization'] =docs.apply(lambda x: self.tokenization(x))#get tokens for all documents
    return list(self.data['tokenization'])

  '''
  Third Step to take is to get the tokens to its origin words by limitization
  '''
  def doc_limitizer(self,doc_token):
    wordnet_lemmatizer = WordNetLemmatizer()
    limitized=[]
    for word in doc_token:
      limitized.append(wordnet_lemmatizer.lemmatize(word))
    return limitized

  def limitization(self,tokens):
    self.data['limitizers'] =tokens.apply(lambda x: self.doc_limitizer(x))#get limitization for all documents
    return list(self.data['limitizers'])

  def get_data(self):
    return self.data

In [ ]:
preprocessing=Text_Preprocessing(df)
cleaned_df=preprocessing.get_data()

In [ ]:
training_data=pd.DataFrame(cleaned_df)

In [ ]:
X_test=[cleaned_df.iloc[1].clean_text,cleaned_df.iloc[2].clean_text]
y_test=[cleaned_df.iloc[1].label,cleaned_df.iloc[2].label]

In [ ]:
cleaned_df

,docs,label,clean_text,tokenization,limitizers
0,comparative politics is a field in political s...,Politics,comparative politics field political science c...,"[comparative, politics, field, political, scie...","[comparative, politics, field, political, scie..."
1,this glossary of history is a list of definiti...,History,glossary history list definitions terms concep...,"[glossary, history, list, definitions, terms, ...","[glossary, history, list, definition, term, co..."
2,a substitute is a political candidate who is n...,Politics,substitute political candidate directly electe...,"[substitute, political, candidate, directly, e...","[substitute, political, candidate, directly, e..."
3,the landscape crisis of the guia fortress refe...,History,landscape crisis guia fortress refers landscap...,"[landscape, crisis, guia, fortress, refers, la...","[landscape, crisis, guia, fortress, refers, la..."
4,a diplomatic mission or foreign mission is a g...,Politics,diplomatic mission foreign mission group peopl...,"[diplomatic, mission, foreign, mission, group,...","[diplomatic, mission, foreign, mission, group,..."
5,health security is a concept that encompasses ...,Politics,health security concept encompasses activities...,"[health, security, concept, encompasses, activ...","[health, security, concept, encompasses, activ..."
6,politics is the process by which groups of peo...,Politics,politics process groups people make decisions ...,"[politics, process, groups, people, make, deci...","[politics, process, group, people, make, decis..."
7,"a biography, or simply bio, is a detailed desc...",History,biography simply bio detailed description pers...,"[biography, simply, bio, detailed, description...","[biography, simply, bio, detailed, description..."
8,"a term of office, electoral term, or parliamen...",Politics,term office electoral term parliamentary term ...,"[term, office, electoral, term, parliamentary,...","[term, office, electoral, term, parliamentary,..."
9,political modernization (also spelled as polit...,Politics,political modernization also spelled political...,"[political, modernization, also, spelled, poli...","[political, modernization, also, spelled, poli..."


In [ ]:
X=training_data.drop(1)
X=training_data.drop(2)

In [ ]:
X=np.array(training_data.tokenization)
y=np.array(training_data.label)
####
X_=list(training_data.tokenization)
y_=list(training_data.label)

# Naive Bayesian model

In [ ]:
from traitlets.traitlets import List
import itertools
from itertools import chain
class Naive_Bayesian:
  '''
  Params:
    origin_docs: dataframe has the following cols: (docs|label|clean_text|tokenization|limitizers)
    X: list of list of docs tokens
    y:list of labels
  '''
  def class_prob(self,cls):
    return self.y.count(cls)/len(self.y)

  def __init__(self,origin_docs,X,y):
    self.X=X
    self.y=y
    self.origin_docs=origin_docs
    self.classes= list(set(y))
    # print(self.classes)
    self
    self.cls_probs=dict() #{cls:cls_prob}
    for cls in self.classes:
      self.cls_probs[cls]=self.class_prob(cls)

    self.all_docs_tokens=list(set(list(chain(*X))))

    #Training
    self.words_probs=dict() #{(c1,word):prob,(c2,word):prob...}
    for cls in self.classes:
      for word in self.all_docs_tokens:
        self.words_probs[(cls,word)]=self.word_prob_given_cls(word,cls)

  def word_prob_given_cls(self,word,cls):
    # 1-Extract cls docs
    cls_docs=list(self.origin_docs.clean_text[self.origin_docs.label==cls])
    concate_docs_words = []
    for doc in cls_docs:
      concate_docs_words+=doc.split()
    cls_docs_unique_words = len(set(concate_docs_words))
    k=len(set(chain(*X)))#[[1,2],[4,6]]==>[1,2,4,6]
    word_count=concate_docs_words.count(word)
    numerator=word_count+1 #laplacian Smoothing numerator
    denominator=cls_docs_unique_words+k
    return (numerator/denominator)

  def get_saved_weights(self):
    return self.words_probs

  def doc_prob_given_class(self,doc,cls,weights):
    doc_words=list(set(doc.split()))
    cls_word_prob=dict()
    cls_probs=[]
    for word in doc_words:
      cls_word_prob[(cls,word)]=self.word_prob_given_cls(word,cls)
    result=1
    probs=[x for x in cls_word_prob.values()]
    doc_given_cls_prob=probs
    for x in doc_given_cls_prob:
      result*=x
    return x

In [ ]:
NB=Naive_Bayesian(training_data,X_,y_)

# Saving words probabilities

In [ ]:
words_probs=NB.get_saved_weights()

In [ ]:
import pickle
# save dictionary to words_probs.pkl file
with open('words_probs.pkl', 'wb') as wp:
  pickle.dump(words_probs, wp)
  print('dictionary saved successfully to file')

dictionary saved successfully to file


# Load Probs and Test it on 2 Docs

In [ ]:
X_test_tokens=[cleaned_df. iloc[1].tokenization,cleaned_df.iloc[2].tokenization]

In [ ]:
# Read dictionary pkl file
with open('words_probs.pkl', 'rb') as wp:
  wpdict = pickle.load(wp)
  print(wpdict)

{('History', 'hitler'): 0.0005421425473471158, ('History', 'elections'): 0.000252999855428654, ('History', 'prowess'): 7.228567297961544e-05, ('History', 'rising'): 0.0003252855284082695, ('History', 'deputy'): 3.614283648980772e-05, ('History', 'amphitheater'): 7.228567297961544e-05, ('History', 'anger'): 7.228567297961544e-05, ('History', 'centuriesthe'): 7.228567297961544e-05, ('History', 'dnvpother'): 3.614283648980772e-05, ('History', 'prefer'): 7.228567297961544e-05, ('History', 'stalin'): 0.00010842850946942316, ('History', 'gastald'): 3.614283648980772e-05, ('History', 'uncovering'): 0.00010842850946942316, ('History', 'implies'): 7.228567297961544e-05, ('History', 'harm'): 0.00014457134595923088, ('History', 'dale'): 7.228567297961544e-05, ('History', 'nineteensixties'): 7.228567297961544e-05, ('History', 'madaxweyne'): 7.228567297961544e-05, ('History', 'change'): 0.002349284371837502, ('History', 'pacifistic'): 7.228567297961544e-05, ('History', 'kilometers'): 7.228567297961

In [ ]:
DocNumber=1
for doc_tokens in X_test_tokens:
  words_probs=[]
  doc_prob=dict()
  for cls in NB.classes:
    for word in set(doc_tokens):
      if (cls,word) in wpdict:
        prob=wpdict[(cls,word)]

        words_probs.append(prob)
    log_sum = sum(np.log2(prob) for prob in words_probs)
    doc_prob[(cls,'doc{}'.format(DocNumber))]=log_sum
  cls_pred=[k for k, v in doc_prob.items() if v == max(doc_prob.values())]
  print(doc_prob)
  print(cls_pred)
  DocNumber+=1
# clss

{('History', 'doc1'): -20940.551132420682, ('Politics', 'doc1'): -44341.590356296605}
[('History', 'doc1')]
{('History', 'doc2'): -1962.7492818292212, ('Politics', 'doc2'): -3831.6255520560253}
[('History', 'doc2')]


In [ ]:
y_test

['History', 'Politics']